# 🤔 **Fine-tunning a NER model with BERT for Beginners**
---

Are you a beginner? Do you want to learn, but don't know where to start? In this tutorial, you will learn to fine-tune a pre-trained BERT model for Named Entity Recognition. It will walk you through the following steps:

- 🚀 Load your training dataset into Argilla and explore it using its tools.
- ⏳ Preprocess the data to generate the other inputs required by the model, and put them in a format that the model expects.
- 🔍 Download the BERT model and start to fine-tune it.
- 🧪 Perform your own tests!

<img src="https://drive.google.com/uc?id=1XUA1RS8p_ARxvtQGTvnoeuDF-JDBfBxv"
  alt="NER"
  width="700"
  height="400"
  style="display: block; margin: 0 auto" />

## Introduction
---

Our goal is to show from a trainig dataset how to fine-tune a **tiny BERT model** in order to identify **NER** tags.

For this purpose, we will first connect to Argilla and log our [dataset](https://huggingface.co/datasets/argilla/spacy_sm_wnut17), so that we can analyse it in a more visual way.

>💡 **Tip:** If you want to try with a different dataset than the one in this tutorial, but it's not yet annotated, Argilla has several tutorials on how to do it [manually](https://docs.argilla.io/en/latest/guides/how_to.html#1.-Manual-labeling) or [automatically](https://docs.argilla.io/en/latest/tutorials/notebooks/labelling-tokenclassification-spacy-pretrained.html#Appendix:-Log-datasets-to-the-Hugging-Face-Hub).


Next, we will preprocess our dataset and fine-tune the model. Here we will be using [DistilBERT](https://huggingface.co/docs/transformers/model_doc/distilbert), to make it easier to understand it and start *playing* with the parameters easily. However, there are still plenty of similar ones to [discover](https://huggingface.co/docs/transformers/index#bigtable).

✨Let's get started!

## Running Argilla
---

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:

1. [Deploy Argilla on Hugging Face Spaces](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla): This is the fastest option and the recommended choice for connecting to external notebooks (e.g., Google Colab) if you have an account on Hugging Face.

<p align="center">
  <a href="https://huggingface.co/new-space?template=argilla/argilla-template-space">
    <img src="https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg" alt="deploy on spaces">
  </a>
</p>

2. [Launch Argilla using Argilla's quickstart Docker image](../../getting_started/quickstart.ipynb): This is the recommended option if you want Argilla running on your local machine. Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

>🤯 **Tip**

> This tutorial is a Jupyter Notebook. There are two options to run it:
- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter notebook tool of your choice.
</div>


## Setup
---

For this tutorial, you'll need to install the Argilla client and a few third party libraries using `pip`:

In [1]:
%pip install "argilla[server]==1.5.0" -qqq
%pip install datasets
%pip install transformers
%pip install evaluate
%pip install seqeval
%pip install transformers[torch]
%pip install accelerate -U

Let's import the Argilla module for reading and writing data:

In [2]:
import argilla as rg

If you are running Argilla using the Docker quickstart image or Hugging Face Spaces, you need to init the Argilla client with the `URL` and `API_KEY`:

In [ ]:
# Replace api_url with the url to your HF Spaces URL if using Spaces (in the three dots in the right-hand corner choose "Embed this Space")
# Replace api_key if you configured a custom API key

# rg.init(
#     api_url="http://localhost:6900",
#     api_key="team.apikey"
# )

rg.init(
    api_url="https://sdiazlor-ner-task.hf.space",
    api_key="team.apikey"
)

Finally, let's include the imports we need:

In [3]:
import pandas as pd
import random
import evaluate
import transformers
import numpy as np
import torch
import pickle

from datasets import load_dataset, ClassLabel, Sequence
from argilla.metrics.token_classification import top_k_mentions
from argilla.metrics.token_classification.metrics import Annotations
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification, pipeline

## 🚀 Exploring our dataset
---

First, we will load the train of our dataset from HuggingFace in order to explore it using ``load_dataset``. And, as we can see, it has 119 entries and two columns: one with the sequence of tokens and the other with the sequence of NER tags.

In [4]:
dataset = load_dataset("argilla/spacy_sm_wnut17", split = "train")

In [5]:
dataset

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 119
})

Next, we will use the following code taking advantage of the ``DatasetDict`` option ``Features`` to convert it to the format required by Argilla in order to log it.

The three elements that our data must have for Token Classifications are the following:

* **text**: the complete string.
* **tokens**: the sequence of tokens.
* **annotation**: a tuple formed by the tag, the start position and the end position.

> ⚠️ **Be careful:** Each execution will upload and add your annotations again without being overwritten.

In [ ]:
def parse_entities(record):
    entities = []
    counter = 0
    for i in range(len(record["ner_tags"])):
        entity = (
            dataset.features["ner_tags"].feature.names[record["ner_tags"][i]],
            counter,
            counter + len(record["tokens"][i]),
        )
        entities.append(entity)
        counter += len(record["tokens"][i]) + 1
    return entities

In [ ]:
# Write a loop to iterate over each row of your dataset and add the text, tokens, and tuple
records = [
    rg.TokenClassificationRecord(
        text=" ".join(row["tokens"]),
        tokens=row["tokens"],
        annotation=parse_entities(row),
    )
    for row in dataset
]

# Log the records with the name of your choice
rg.log(records, "spacy_sm_wnut17")

Logging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

119 records logged to ]8;id=254486;https://sdiazlor-ner-task.hf.space/datasets/team/spacy_sm_wnut17\https://sdiazlor-ner-task.hf.space/datasets/team/spacy_sm_wnut17]8;;\

BulkResponse(dataset='spacy_sm_wnut17', processed=119, failed=0)

So now you will be able to check your annotations in a much more visual way and even edit them if necessary.

<img src="https://drive.google.com/uc?id=191wpFz5MsQtiJNFeH9IXizr65qtGE3MP"
  alt="NER"
  width="1500"
  height="400"
  style="display: block; margin: 0 auto" />

In addition, **Argilla** also has more options, e.g. to extract [metrics](https://docs.argilla.io/en/latest/reference/python/python_metrics.html) such as the ones shown below.



In [ ]:
# Select the dataset from Argilla and visualize the data
top_k_mentions(
    name="spacy_sm_wnut17", k=30, threshold=2, compute_for=Annotations
).visualize()

## ⏳ Preprocessing the data
---

Next, we will **pre-process our data** in the required format so that the model can work with it. In our case, we will reload them from HuggingFace, as in Argilla we only loaded the train set, however, this is also possible.

The following code would allow us to prepare our data using Argilla, this is especially useful for manual annotations, as it adds **B-** (beggining) or **I-** (inside) to our NER tags automatically depending on their position.

```python
dataset = rg.load("dataset_name").prepare_for_training()

dataset = dataset.train_test_split()
```

> 🤯 **Tip:** In our case, we are working with a very small dataset that is divided into train and test. However, you may are using another dataset that already have the ``validation`` partition, or even if it is larger, you can create this partition yourself with the following code:

```python
dataset['train'], dataset['validation'] = dataset['train'].train_test_split(.1).values()
```
So, let's continue!

In [6]:
dataset = load_dataset("argilla/spacy_sm_wnut17")
print(dataset)

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 119
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 30
    })
})


Time to **tokenize**! Although it seems like this is already done, each token still needs to be converted into a vector (ID) that the model can read from its pretrained vocabulary. To do this, we will use ``AutoTokenizer.from_pretrained`` and the FastTokenizer ``distilbert-base-uncased``.

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [8]:
# Example of original tokens
example = dataset["train"][0]
print(example["tokens"])

# Example after executing the AutoTokenizer
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['says', 'it', "'s", 'Saturday', '!', 'I', "'m", 'wearing', 'my', 'Weekend', '!', ':)']
['[CLS]', 'says', 'it', "'", 's', 'saturday', '!', 'i', "'", 'm', 'wearing', 'my', 'weekend', '!', ':', ')', '[SEP]']


However, we now ran into a new problem. Since it tokenises according to a pre-trained vocabulary, this will create new subdivisions in some words (e.g. "'s" in "'" and "s"). In addition, it adds two new tags [CLS] and [SEP]. Therefore, we have to realign the IDs of our words with the corresponding NER tags, thanks to the ``word-ids`` method.

In [9]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

## 🔍 Fine-tunning the model
---
We should now start preparing the parameters of our model, i.e. we should start fine-tuning.

### Model

First we will download our pretrained model with ``AutoModelForTokenClassification`` and we will indicate the name of the chosen model, the number of tags and we will indicate the correspondences between their IDs and their names.

In addition, we will also set our ``DataCollator`` to form a batch by using our processed examples as input. In this case, we will be using ``DataCollatorForTokenClassification``.

In [10]:
# Create a dictionary with the ids and the relevant label.
label_list = dataset["train"].features["ner_tags"].feature.names
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

# Download the model.
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id)

# Set the DataCollator
data_collator = DataCollatorForTokenClassification(tokenizer)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

### Training arguments
The ``TrainingArguments`` class will include the [parameters](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) to customise our training.

> 💡 **Tip:** If you are using HuggingFace it may be easier for you to save your model there directly. To do so, use the following code and add the following parameters to TrainingArguments.

```python
from huggingface_hub import notebook_login
notebook_login()

# Add the following parameter
training_args = TrainingArguments(
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)
```
> 🕹️ **Let's play:** What is the best accuracy you can get?

In [11]:
training_args = TrainingArguments(
    output_dir="ner-recognition",
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    weight_decay=0.05,
    evaluation_strategy="epoch",
    optim="adamw_torch",
    logging_steps = 50
)

### Metrics

To know how our training has gone, of course, we must use metrics. Therefore, we will use ``Seqeval`` and a function that computes precision, recall, F1 and accuracy from the actual and predicted tags.

In [12]:
# Load Sqeval.
metric = evaluate.load("seqeval")

# Create the list with the tags.
labels = [label_list[i] for i in example[f"ner_tags"]]

# Function to compute precision, recall, F1 and accuracy.
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Time to train

As the name of this section suggests, the time has come to bring all the previous elements together and start training with ``Trainer``.

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train.
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.445835,0.000000,0.000000,0.000000,0.720751
2,No log,1.540381,0.000000,0.000000,0.000000,0.720751
3,No log,1.300941,0.000000,0.000000,0.000000,0.720751
4,No log,1.259119,0.000000,0.000000,0.000000,0.720751
5,No log,1.256542,0.444444,0.025478,0.048193,0.720751
6,No log,1.154050,0.202703,0.095541,0.129870,0.736203
7,No log,1.388463,0.254545,0.089172,0.132075,0.718543
8,No log,1.246235,0.275362,0.121019,0.168142,0.737307
9,No log,1.254787,0.202020,0.127389,0.156250,0.731788
10,No log,1.388549,0.272727,0.171975,0.210938,0.735099


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=80, training_loss=0.45428856909275056, metrics={'train_runtime': 14.9864, 'train_samples_per_second': 158.811, 'train_steps_per_second': 5.338, 'total_flos': 32769159790410.0, 'train_loss': 0.45428856909275056, 'epoch': 20.0})

The `evaluate` method will allow you to evaluate again on the validation set or on another dataset (e.g. if you have train, validation and test).

In [14]:
trainer.evaluate()

{'eval_loss': 1.5393034219741821,
 'eval_precision': 0.2773109243697479,
 'eval_recall': 0.21019108280254778,
 'eval_f1': 0.2391304347826087,
 'eval_accuracy': 0.7450331125827815,
 'eval_runtime': 0.0918,
 'eval_samples_per_second': 326.934,
 'eval_steps_per_second': 10.898,
 'epoch': 20.0}

> 🔮 **Try to predict**

> When you have created your model and are happy with it, test it yourself with your own text.

```python
# Replace this with the directory where it was saved
model_checkpoint = "your-path"
token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")
token_classifier("I heard Madrid is wonderful in spring.")

```

## 📝✔️ Summary

In this tutorial, we have learned how to upload our training dataset to Argilla in order to visualise the data it contains and the NER tags it uses and how to fine-tune a BERT model for NER recognition using ``transformers``. This can be very useful to learn the basics of BERT pre-models and, from there, to develop your skills further and try out different ones that may give better results.

💪Cheers!